# EfficientNet Preprocessing Investigation

This notebook investigates what EfficientNet's `preprocess_input` function actually does to understand if we need mean/std normalization.

In [ ]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import matplotlib.pyplot as plt

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Load Dataset - Create a simple test image
# Create a test image array [224, 224, 3] with values in [0, 255] range
test_image_255 = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
test_image_float = test_image_255.astype(np.float32)

print(f"Test image shape: {test_image_255.shape}")
print(f"Test image dtype: {test_image_255.dtype}")
print(f"Test image range: {test_image_255.min()} to {test_image_255.max()}")

In [ ]:
# Check Preprocessing Steps - Test what preprocess_input actually does

# Method 1: Direct preprocessing on [0,255] range
print("=== Method 1: Direct preprocess_input on [0,255] ===")
processed_255 = preprocess_input(test_image_float.copy())
print(f"Input range: {test_image_float.min():.3f} to {test_image_float.max():.3f}")
print(f"Output range: {processed_255.min():.3f} to {processed_255.max():.3f}")
print(f"Output mean: {processed_255.mean():.3f}")
print(f"Output std: {processed_255.std():.3f}")

# Method 2: Normalize to [0,1] first, then preprocess_input
print("\n=== Method 2: Normalize to [0,1] then preprocess_input ===")
normalized_01 = test_image_float / 255.0
processed_from_01 = preprocess_input(normalized_01 * 255.0)
print(f"Input range: {normalized_01.min():.3f} to {normalized_01.max():.3f}")
print(f"After *255: {(normalized_01 * 255.0).min():.3f} to {(normalized_01 * 255.0).max():.3f}")
print(f"Output range: {processed_from_01.min():.3f} to {processed_from_01.max():.3f}")
print(f"Output mean: {processed_from_01.mean():.3f}")
print(f"Output std: {processed_from_01.std():.3f}")

# Method 3: Just normalize to [0,1] (no preprocess_input)
print("\n=== Method 3: Simple [0,1] normalization only ===")
simple_normalized = test_image_float / 255.0
print(f"Output range: {simple_normalized.min():.3f} to {simple_normalized.max():.3f}")
print(f"Output mean: {simple_normalized.mean():.3f}")
print(f"Output std: {simple_normalized.std():.3f}")

In [ ]:
# Verify Mean and Standard Deviation Usage
# Let's test with a known image to see exactly what preprocess_input does

# Create test image with known values
test_values = np.array([[[100, 150, 200]]], dtype=np.float32)  # Single pixel
print(f"Original pixel values: {test_values[0,0]}")

# Apply preprocess_input
processed_values = preprocess_input(test_values.copy())
print(f"After preprocess_input: {processed_values[0,0]}")

# Manual calculation to see if it matches ImageNet normalization
# ImageNet mean: [123.675, 116.28, 103.53]
# ImageNet std: [58.395, 57.12, 57.375]
imagenet_mean = np.array([123.675, 116.28, 103.53])
imagenet_std = np.array([58.395, 57.12, 57.375])

manual_normalized = (test_values[0,0] - imagenet_mean) / imagenet_std
print(f"Manual ImageNet normalization: {manual_normalized}")

# Test if they match
matches_imagenet = np.allclose(processed_values[0,0], manual_normalized, rtol=1e-3)
print(f"Does preprocess_input match ImageNet normalization? {matches_imagenet}")

# Test if it's just [0,1] normalization  
simple_norm = test_values[0,0] / 255.0
matches_simple = np.allclose(processed_values[0,0], simple_norm, rtol=1e-3)
print(f"Does preprocess_input match simple [0,1] normalization? {matches_simple}")

## Conclusion

This notebook tests what EfficientNet's `preprocess_input` function actually does:

1. **If it uses ImageNet normalization**: (pixel - mean) / std
2. **If it's just simple normalization**: pixel / 255.0  
3. **Or something else entirely**

The results will help us understand why our Flutter app classifies everything as cardboard!